## Install Library

Install hnswlib terlebih dahulu.

In [1]:
!pip install hnswlib

## Percobaan

Percobaan berikut akan membandingkan exact NN dengan HNSW pada 1000 data 2D.

In [2]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ===========================
# 1. Buat data 2D acak
# ===========================
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

# ===========================
# 2. Exact NN (Brute Force)
# ===========================
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

# ===========================
# 3. HNSW
# ===========================
# Inisialisasi index HNSW
p = hnswlib.Index(space='l2', dim=dim)  

# Ukuran maksimum elemen yang bisa ditampung
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data
p.add_items(data)

# Set parameter pencarian
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

=== Exact NN ===
Indices: [[181  28 275 302 456]]
Distances: [[0.0146186  0.01828245 0.0222434  0.0273031  0.03661864]]
Waktu: 0.15984272956848145 detik

=== HNSW ===
Indices: [[181  28 275 302 456]]
Distances: [[0.0002137  0.00033425 0.00049477 0.00074546 0.00134092]]
Waktu: 0.0007090568542480469 detik


## Tugas

Lakukan percobaan pada metric distance yang berbeda, 1000 vs 1jt data, 2D vs 5D data. Catat hasilnya pada tabel yang anda buat sendiri seperti pada praktikum 1.